## Sizing

In [1]:
import pint
ureg = pint.get_application_registry()
ureg.default_format = ".2f~P"
Q_ = pint.Quantity
Q_.default_format = ".2f~P"

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np       
import pandas as pd
from scipy.optimize import root_scalar                               
!pip install cedar
from cedar import Plant

Looking in indexes: https://pypi.org/simple, https://asia-southeast1-python.pkg.dev/kindling-2306/python/simple/


In [3]:
%%capture
import subprocess, importlib
if importlib.util.find_spec('import-ipynb') is None:
    !pip install import-ipynb

import import_ipynb
from RFQ_Load import * 
from RFQ_MEB import *

In [4]:
#bin calculation
from math import pi, sin, cos, tan, sqrt

In [5]:
rho_steel = 7950 * ureg.kg/ureg.m**3 
rho_carbon_steel = 7800 * ureg.kg/ureg.m**3

In [6]:
h_vap=Q_(2650,ureg.kJ/ureg.kg)
m_ = Q_(1326, ureg.kg/ureg.hr)
cp = Q_(1.5, ureg.kJ/ureg.kg/ureg.K)
dT = Q_(200, ureg.K)

m_*cp*dT/h_vap

150.11320754716982 <Unit('kilogram / hour')>

## Kiln 

In [ ]:
import math
#############################################################################
############
###HIDDEN###
############
#############################################################################
T_op = 750 * ureg.degC
T_in = 25 * ureg.degC
ym = 0.1112
rhogas = 1 * ureg.atm * (20.3 * ureg.g/ureg.mol) / (8.314 * ureg.J/ureg.K/ureg.mol * T_op)
#############################################################################
############
###HIDDEN###
############
#############################################################################
Lar = S[201].L
ymoist = design.feed.moisture
h_gross =  (-S[601].HHV / design.feed.nominal_feed_rate).to(ureg.kW * ureg.hr / ureg.kg)
m_load = (design_flux * kosa / h_gross).to_reduced_units()
#############################################################################
############
###HIDDEN###
############
#############################################################################
############
###HIDDEN###
############
#############################################################################
minval = min(mgas, mbed, m_load, mchoke)

mkiln = 0
if minval == mgas:
############
###HIDDEN###
############
elif minval == mbed:
############
###HIDDEN###
############
elif minval == m_load:
############
###HIDDEN###
############
elif minval == mchoke:
############
###HIDDEN###
############

print(design_flux)

print(f"Kiln(s) required: {design.feed.nominal_feed_rate/mkiln}") 

250.00 kg/m³ 200.00 kg/m³
2100.00 mm 10500.00 mm 8.00 mm 2150.00 mm 9770.00 mm 2960.00 mm
mkiln = mload = 3007.21 kg/h
40.00 kW/m²
Kiln(s) required: 2.08


In [8]:
def get_parameter(name, kwargs, plant, default=None):
    return kwargs.get(name, getattr(plant, name, default))

def bin_volume_calc(bin, **kwargs):
    diameter = get_parameter('diameter', kwargs, bin)
    dia_inlet = get_parameter('dia_inlet', kwargs, bin)
    dia_outlet = get_parameter('dia_outlet', kwargs, bin)
    theta_rill = get_parameter('theta_rill', kwargs, bin)
    theta_cone = get_parameter('theta_cone', kwargs, bin)
    h_total = get_parameter('h_total', kwargs, bin)
    h_throat = get_parameter('h_throat', kwargs, bin)
    
    r = diameter / 2
    r_in = dia_inlet / 2
    r_out = dia_outlet / 2

    h_rill_cone = (diameter - dia_inlet) / 2 * tan(theta_rill)
    h_bottom_cone = (diameter - dia_outlet) / 2 * tan(theta_cone)

    h_cylinder = h_total - 2 * h_throat - h_bottom_cone

    h = h_cylinder + h_bottom_cone
    area = (pi * r ** 2).to(ureg.m ** 2)
    volume_bottom_cone = (h_bottom_cone / 3 * pi * (r ** 2 + r * r_out + r_out ** 2)).to(ureg.m ** 3)
    volume_cylinder_full = (h_cylinder * area).to(ureg.m**3)
    volume_rill_cone = (h_rill_cone/3*pi* (r ** 2 + r * r_in + r_in ** 2)).to(ureg.m**3)
    volume_cylinder_limited = ((h_cylinder-h_rill_cone) * area).to(ureg.m**3)
    volume_live = volume_bottom_cone + volume_cylinder_limited + volume_rill_cone
    volume_max = volume_bottom_cone + volume_cylinder_full
    return (volume_live, volume_max, h_cylinder)


def bin_internal_calc(bin, **kwargs):
    diameter = get_parameter('diameter', kwargs, bin)
    dia_inlet = get_parameter('dia_inlet', kwargs, bin)
    dia_outlet = get_parameter('dia_outlet', kwargs, bin)
    theta_rill = get_parameter('theta_rill', kwargs, bin)
    theta_cone = get_parameter('theta_cone', kwargs, bin)
    h_cylinder = get_parameter('h_cylinder', kwargs, bin)
    
    r = diameter / 2
    r_in = dia_inlet / 2
    r_out = dia_outlet / 2

    h_bottom_cone = (diameter - dia_outlet) / 2 * tan(theta_cone)
    area = (pi * r ** 2).to(ureg.m ** 2)
    volume_bottom_cone = (h_bottom_cone / 3 * pi * (r ** 2 + r * r_out + r_out ** 2)).to(ureg.m ** 3)
    volume_cylinder_full = (h_cylinder * area).to(ureg.m**3)
    volume_max = volume_bottom_cone + volume_cylinder_full
    return (volume_max)

In [ ]:
BN202 = Plant("BN-202 Feed #1") 
BN202internal = Plant("BN-202 Feed #1 internal")

with BN202 as b, BN202internal as bi: 
############
###HIDDEN###
############

#print(BN201)
print(BN202)

========================================== BN-202 Feed #1 ==========================================
 tag                                  : BN-202 Feed #1
 diameter                             : 3,000 mm
 dia_inlet                            : 250 mm
 dia_outlet                           : 220 mm
 theta_rill                           : 30 deg
 theta_cone                           : 50 deg
 h_total                              : 3,500 mm
 h_throat                             : 100 mm
 volume_live                          : 12.26 m³
 volume_max                           : 15.83 m³
 h_cylinder                           : 1,643 mm
 volume_low                           : 10.13 m³
 volume_diff                          : 2.121 m³
 feed_rate                            : 6,250 kg/h
 density                              : 250 kg/m³
 flowrate                             : 25 m³/h
 time_between_fill                    : 5.089 min
 thickness                            : 5 mm
 mass_material_req    

In [10]:
# class ConicalBin:
#     def __init__(self, dia, height, theta_rill, theta_cone, inlet_dia, outlet_dia):
#         self.dia = dia`
#         self.radius = dia/2
#         self.area = 
#         self.height = height

BN311 = Plant("Product holding bin #311")
BN311internal = Plant("BN-311 Loading #1 internal")

with BN311 as bn, BN311internal as bni: 
    bn.diameter = 1600 * ureg.mm 
    bn.dia_inlet = 226*ureg.mm 
    bn.dia_outlet = 200*ureg.mm # dia q
    bn.theta_rill = 35*ureg.deg #keep the same since repose is 23 also 
    bn.theta_cone = 55*ureg.deg #keep the same since repose is 23 also 
    bn.h_total = Q_(2200, ureg.mm) # includes throat dimensions 
    bn.h_throat = Q_(100, ureg.mm) 
    bn.volume_live, bn.volume_max, bn.h_cylinder = bin_volume_calc(bn) 
    bn.volume_low, _, _ = bin_volume_calc(bn, h_total=Q_(1800, ureg.mm)) 
    bn.volume_diff = bn.volume_live - bn.volume_low 
    bn.feed_rate = S[305].m
    bn.density = design.feed.bulk_density 
    bn.flowrate = bn.feed_rate / bn.density 
    bn.time_to_fill = (bn.volume_diff / bn.flowrate).to(ureg.min) 
    bn.thickness = 5 * ureg.mm 
    
    bni.diameter = bn.diameter - bn.thickness*2
    bni.dia_inlet = 226*ureg.mm
    bni.dia_outlet = 200*ureg.mm
    bni.h_cylinder = bn.h_cylinder
    bni.theta_rill = 35*ureg.deg
    bni.theta_cone = 55*ureg.deg
    bni.vol_internal_max = bin_internal_calc(bni)
    bn.mass_material_req = (bn.volume_max - bni.vol_internal_max)*rho_carbon_steel

bn = None
bni = None
print(BN311)
#print(BN311internal)

===================================== Product holding bin #311 =====================================
 tag                                  : Product holding bin #311
 diameter                             : 1,600 mm
 dia_inlet                            : 226 mm
 dia_outlet                           : 200 mm
 theta_rill                           : 35 deg
 theta_cone                           : 55 deg
 h_total                              : 2,200 mm
 h_throat                             : 100 mm
 volume_live                          : 2.183 m³
 volume_max                           : 2.775 m³
 h_cylinder                           : 1,000 mm
 volume_low                           : 1.378 m³
 volume_diff                          : 0.8042 m³
 feed_rate                            : 1,034 kg/h
 density                              : 250 kg/m³
 flowrate                             : 4.138 m³/h
 time_to_fill                         : 11.66 min
 thickness                            : 5 mm
 mass_ma

## Screw Conveyor 



In [ ]:
v_101 = Q_(6, ureg.m**3/ureg.hr)

def auger_capacity(dia, hub, pitch, thickness, speed, fill):
    """
    Dia should be in mm
    hub should be in mm
    pitch in mm
    output is capacity of auger
    """
    v_rad = (pi/4*(dia**2-hub**2)*(pitch-thickness)/2/pi).to(ureg.L/ureg.rad)
    return (v_rad*speed*fill).to(ureg.m**3/ureg.hr)


CV204 = Plant("CV-204 screw conveyor")
with CV204 as c:
############
###HIDDEN###
############
    c.auger_capacity = auger_capacity(c.dia, c.hub, c.pitch, c.thickness, c.speed, c.fill_cap)

print(CV204)


====================================== CV-204 screw conveyor =======================================
 tag                                  : CV-204 screw conveyor
 mass                                 : 200 kg
 dia                                  : 210 mm
 hub                                  : 50 mm
 pitch                                : 200 mm
 thickness                            : 5 mm
 speed                                : 18 rpm
 fill_cap                             : 0.95
 auger_capacity                       : 6.537 m³/h



## Thermal Expansion of Heat Tube

In [ ]:
Heat_Tube = Plant("Kiln Heat Tube")
with Heat_Tube as h:
############
###HIDDEN###
############

print(h)

========================================== Kiln Heat Tube ==========================================
 tag                                  : Kiln Heat Tube
 coeff                                : 1.9×10⁻⁵ 1/K
 T_op                                 : 900 °C
 dT                                   : 875.2 K
 original_length_cold                 : 8,000 mm
 expansion                            : 133 mm



In [13]:
## Fan 
egfr = S[210]

FN208 = Plant("FN-208 exhaust fan")
with FN208 as f:
    f.exhaust_gas_flow_rate = egfr.v
    f.exhaust_gas_p = egfr.P

print(FN208)

======================================== FN-208 exhaust fan ========================================
 tag                                  : FN-208 exhaust fan
 exhaust_gas_flow_rate                : 5.033×10⁴ m³/h
 exhaust_gas_p                        : 101.3 kPa



## Cooling Conveyors 

In [14]:
CV306 = Plant("CV-306")
with CV306 as c1:
    c1.fluid = "cooling water"
    c1.design_capacity_mass = S[203].m
    c1.design_capacity = S[203].v
    c1.outer_diameter = units["CV-306"].outer_diameter
    c1.length = units["CV-306"].length
    c1.inner_ID = units["CV-306"].inner_ID
    c1.inner_OD = units["CV-306"].inner_OD
    c1.outer_ID = units["CV-306"].outer_ID
    c1.outer_OD = units["CV-306"].outer_OD
    x1 = np.pi/4 * (c1.inner_OD)**2 * c1.length
    x2 = np.pi/4 * (c1.inner_ID)**2 * c1.length
    y1 = np.pi/4 * (c1.outer_OD)**2 * c1.length
    y2 = np.pi/4 * (c1.outer_ID)**2 * c1.length
    x = x1 - x2 
    y = y1 - y2
    c1.material = "stainless steel"
    c1.mass = ((x + y)*rho_steel).to(ureg.kg)
    
c1 = None

CV307, CV308 = Plant("CV-307"), Plant("CV-308")
with CV307 as c1, CV308 as c2:
    c2.fluid = c1.fluid = "cooling water"
    c2.design_capacity_mass = c1.design_capacity_mass = S[203].m
    c2.design_capacity = c1.design_capacity = S[203].v
    c2.outer_diameter , c1.outer_diameter = units["CV-308"].outer_diameter, units["CV-307"].outer_diameter
    c2.length, c1.length = units["CV-308"].length, units["CV-307"].length
    c2.inner_ID, c1.inner_ID = units["CV-308"].inner_ID, units["CV-307"].inner_ID
    c2.inner_OD,  c1.inner_OD = units["CV-308"].inner_OD, units["CV-307"].inner_OD
    c2.outer_ID,  c1.outer_ID = units["CV-308"].outer_ID, units["CV-307"].outer_ID
    c2.outer_OD,  c1.outer_OD = units["CV-308"].outer_OD, units["CV-307"].outer_OD
    c2.material, c1.material = "carbon steel", "carbon steel"
    x1 = np.pi/4 * (c1.inner_OD)**2 * c1.length
    x2 = np.pi/4 * (c1.inner_ID)**2 * c1.length
    y1 = np.pi/4 * (c1.outer_OD)**2 * c1.length
    y2 = np.pi/4 * (c1.outer_ID)**2 * c1.length
    x = x1 - x2 
    y = y1 - y2
    c1.material = "stainless steel"
    c2.mass = c1.mass = ((x + y)*rho_steel).to(ureg.kg)
   



print(CV306)
print(CV307)
print(CV308)

============================================== CV-306 ==============================================
 tag                                  : CV-306
 fluid                                : cooling water
 design_capacity_mass                 : 1,034 kg/h
 design_capacity                      : 5.305 m³/h
 outer_diameter                       : 250 mm
 length                               : 3,000 mm
 inner_ID                             : 211.6 mm
 inner_OD                             : 219.1 mm
 outer_ID                             : 227.1 mm
 outer_OD                             : 244.5 mm
 material                             : stainless steel
 mass                                 : 214.2 kg

============================================== CV-307 ==============================================
 tag                                  : CV-307
 fluid                                : cooling water
 design_capacity_mass                 : 1,034 kg/h
 design_capacity                      : 5.305

# Bucket Elevator

Bucket design based on 220mm width and 
- breadth of 150mm
- flange of 30mm
- angle of 30° to 120°
- depth approx 93mm



## 

In [ ]:
############
###HIDDEN###
############

V = (A*w*f).to(ureg.L)
print(V)

############
###HIDDEN###
############

true_tip_speed = omega_sprocket * (sprocket_radius + length/ureg.rad)

tip_accel = (omega_sprocket**2 * (sprocket_radius + length/ureg.rad)).to(ureg.m/ureg.s**2)

print(true_tip_speed.to(ureg.m/ureg.s))
print(omega_sprocket.to(ureg.rpm))
print(tip_accel)

buckets = (u_max/pitch).to(ureg.Hz)



v_max = (buckets*V).to(ureg.m**3/ureg.hr)
print(buckets)
print(v_max)

## HX sizing for scrubber

In [16]:
set_stream(S[4101],
           M = S[410].M + S[13].M + S[14].M,
           T = S[410].T,
           P = S[410].P
          )

## Set-up
############################################################
U = 3209 * ureg.W/ureg.K/ureg.m**2 ## <<-- value that is fixed and set as per vendor recommendations
A_plate = 0.5 * ureg.m**2 ## <<-- value that is fixed and set as per vendor recommendations 
############################################################

cold_side_inlet = S[7]
cold_side_outlet = S[415]
hot_side_inlet = S[4101]
hot_side_outlet = S[411]

#print(cold_side_inlet, cold_side_outlet, hot_side_inlet, hot_side_outlet)

#check heat duties on both end:
hot_side_Q = S[4101].Q - S[411].Q
cold_side_Q = S[415].Q - S[7].Q 
e = 5 * ureg.MJ/ureg.hr

if (cold_side_Q > hot_side_Q) and (cold_side_Q - hot_side_Q < e): 
    t1 = hot_side_inlet.T - cold_side_outlet.T
    t2 = hot_side_outlet.T - cold_side_inlet.T
    lmtd = (t1 - t2)/np.log(t1/t2)
    A = (cold_side_Q / (lmtd * U)).to(ureg.m**2)
    print(f"Area required for HX-605: {A}")
    
    num_plate = A // A_plate
    if A % A_plate != 0: 
        num_plate += 1
    print(f"No. of plates required: {num_plate}")

elif (hot_side_Q > cold_side_Q) and (hot_side_Q - cold_side_Q < e):
    t1 = hot_side_inlet.T - cold_side_outlet.T
    print(t1)
    t2 = hot_side_outlet.T - cold_side_inlet.T
    print(t2)
    lmtd = (t1 - t2)/np.log(t1/t2)
    A = (hot_side_Q / (lmtd * U)).to(ureg.m**2)
    print(f"Area required for HX-405: {A}")
    num_plate = A // A_plate
    if A % A_plate != 0: 
        num_plate += 1
    print(f"No. of plates required: {num_plate}")

## SACTO sizing

In [18]:
AFRcalc = air_to_fuel_ratio(S[205].Y)
v_emg = S[421].v

In [ ]:
############
###HIDDEN###
############

V = (4/pi*dia**2*l).to(ureg.m**3)
V_in = (4/pi*dia_in**2*l).to(ureg.m**3)
V_mat = V - V_in


emg_excess_air = 0.25
AFR = AFRcalc * (1 + emg_excess_air)
excess_air = 1
v_comb_emg = v_emg*((1+AFR)*(1+emg_excess_air))
tau_emg = (V_in/v_comb_emg).to(ureg.s)


print(f"SACTO Volume             = {V}")
print(f"SACTO actual volume      = {V_in}")
print(f"Peak Emergency Gas flow  = {v_emg}")
print(f"peak combustion flow     = {v_comb_emg}")


print(f"======================================")
print(f"residence time emergency = {tau_emg}")
print(f"Ratio                    = {l/dia}")
print(f"Material volume          = {V_mat}")
print(f"Mass of SACTO            = {V_mat*rho_steel}")

SACTO Volume             = 56.15 m³
SACTO actual volume      = 55.72 m³
Peak Emergency Gas flow  = 15603.42 m³/h
peak combustion flow     = 117841.25 m³/h
residence time emergency = 1.70 s
Ratio                    = 4.76
Material volume          = 0.43 m³
Mass of SACTO            = 3394.60 kg
